In [1]:
from datasets import load_dataset
import pandas as pd

/home/jaykold/dev/llm-project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#full_data = load_dataset("squad_v2", split="train")

In [3]:
# shuffled_data = full_data.shuffle(seed=42)

# type(shuffled_data)

In [4]:
# shuffled_data = pd.DataFrame(shuffled_data)

In [5]:
# shuffled_data.head()

In [ ]:
#shuffled_data.to_csv("data/squad_v2.csv")
# data = shuffled_data.head(600)

In [ ]:
# df = pd.DataFrame(data)
# df['title']

0                     Canon_law
1      Athanasius_of_Alexandria
2         Jehovah%27s_Witnesses
3                       Estonia
4               Classical_music
                 ...           
595     Northwestern_University
596                       Paper
597                    Zhejiang
598    Brigham_Young_University
599                     YouTube
Name: title, Length: 600, dtype: object

In [ ]:
# df.to_json("data/squad_v2_600.json", orient="records", indent=2)

In [1]:
import json

with open("data/squad_v2_600.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [2]:
data[11]

{'id': '57325c2fe99e3014001e6716',
 'title': 'Jehovah%27s_Witnesses',
 'context': 'Sociologist Rodney Stark states that Jehovah\'s Witness leaders are "not always very democratic" and that members "are expected to conform to rather strict standards," but adds that "enforcement tends to be very informal, sustained by the close bonds of friendship within the group", and that Jehovah\'s Witnesses see themselves as "part of the power structure rather than subject to it." Sociologist Andrew Holden states that most members who join millenarian movements such as Jehovah\'s Witnesses have made an informed choice. However, he also states that defectors "are seldom allowed a dignified exit", and describes the administration as autocratic.',
 'question': "How do Jehovah's Witnesses see themselves in regards to the power structure of the church?",
 'answers': {'text': ['"part of the power structure rather than subject to it."'],
  'answer_start': [330]}}

In [3]:
from urllib.parse import unquote

def clean_text(value):
    """Decode unicode escapes and URL encodings in a string."""
    if not isinstance(value, str):
        return value
    try:
        # Decode URL-encoded characters (%20, %27, etc.)
        value = unquote(value)
        # Decode unicode escapes (\u00f3 etc.)
        #value = value.encode('utf-8').decode('unicode_escape')
        # Decode Python-style backslash escapes (e.g., \' -> ')
        #value = value.encode('utf-8').decode('unicode_escape')
        # Replace underscores with spaces in certain cases
        value = value.replace('_', ' ')
        return value
    except Exception:
        return value  # Fail silently if decoding fails

def clean_data(obj):
    """Recursively clean all strings in a nested structure."""
    if isinstance(obj, dict):
        return {k: clean_data(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [clean_data(item) for item in obj]
    elif isinstance(obj, str):
        return clean_text(obj)
    else:
        return obj


In [4]:
cleaned_data = clean_data(data)

In [5]:
type(cleaned_data)
cleaned_data[:3]

[{'id': '56e0f3907aa994140058e80a',
  'title': 'Canon law',
  'context': 'The Roman Catholic Church canon law also includes the main five rites (groups) of churches which are in full union with the Roman Catholic Church and the Supreme Pontiff:',
  'question': 'What term characterizes the intersection of the rites with the Roman Catholic Church?',
  'answers': {'text': ['full union'], 'answer_start': [104]}},
 {'id': '571adcf932177014007e9f56',
  'title': 'Athanasius of Alexandria',
  'context': "Alexandria was the most important trade center in the whole empire during Athanasius's boyhood. Intellectually, morally, and politically—it epitomized the ethnically diverse Graeco-Roman world, even more than Rome or Constantinople, Antioch or Marseilles. Its famous catechetical school, while sacrificing none of its famous passion for orthodoxy since the days of Pantaenus, Clement of Alexandria, Origen of Alexandria, Dionysius and Theognostus, had begun to take on an almost secular character i

In [6]:
cleaned_data[11]['answers']['text'][0].replace('"', "")

'part of the power structure rather than subject to it.'

In [7]:
documents = []

for item in cleaned_data:
    # Always include the entry, even if there is no answer
    answer = ""
    if item['answers']['text'] and len(item['answers']['text']) > 0:
        answer = item['answers']['text'][0].replace('"', "")

    documents.append({
        "id": item["id"],
        'title': item['title'],
        "context": item['context'].replace("\n", " "),
        "question": item['question'].replace("\n", " "),
        "answer": answer
    })

In [8]:
documents[10]

{'id': '56fb84ebb28b3419009f1de7',
 'title': 'Middle Ages',
 'context': 'During this period the practice of manuscript illumination gradually passed from monasteries to lay workshops, so that according to Janetta Benton "by 1300 most monks bought their books in shops", and the book of hours developed as a form of devotional book for lay-people. Metalwork continued to be the most prestigious form of art, with Limoges enamel a popular and relatively affordable option for objects such as reliquaries and crosses. In Italy the innovations of Cimabue and Duccio, followed by the Trecento master Giotto (d. 1337), greatly increased the sophistication and status of panel painting and fresco. Increasing prosperity during the 12th century resulted in greater production of secular art; many carved ivory objects such as gaming-pieces, combs, and small religious figures have survived.',
 'question': 'What were many pieces of secular art carved from in this period?',
 'answer': 'ivory'}

In [9]:
# Prepare documents for embedding
# We'll embed the combination of context + question for better retrieval
# This way, even documents without answers can be found when relevant

embedding_documents = []
for doc in documents:
    # Combine context and question for embedding
    # This creates a rich text representation for semantic search
    embedding_text = f"Question: {doc['question']} Context: {doc['context']}"

    embedding_documents.append({
        "id": doc["id"],
        "text": embedding_text,  # This is what we'll embed
        "metadata": {
            "title": doc["title"],
            "context": doc["context"],
            "question": doc["question"],
            "answer": doc["answer"],
            "has_answer": bool(doc["answer"])  # Flag to indicate if answer exists
        }
    })

print(f"Prepared {len(embedding_documents)} documents for embedding")
display(f"Sample embedding text: {embedding_documents[3]['text'][:200]}...")

Prepared 600 documents for embedding


'Sample embedding text: Question: What are the most common foods in Estonia? Context: Historically, the cuisine of Estonia has been heavily dependent on seasons and simple peasant food, which today is influenced by many coun...'

In [10]:
# Check how many documents have empty answers
empty_answers = sum(1 for doc in documents if doc['answer'] == "")
total_docs = len(documents)
print(f"Total documents: {total_docs}")
print(f"Documents with empty answers: {empty_answers}")
print(f"Documents with answers: {total_docs - empty_answers}")


for i, doc in enumerate(documents[3:7]):
    print(f"\nDocument {i+1}:")
    print(f"Question: {doc['question'][:100]}...")
    print(f"Answer: '{doc['answer']}'" if doc['answer'] else "Answer: (empty)")
    print(f"Has answer: {bool(doc['answer'])}")

Total documents: 600
Documents with empty answers: 183
Documents with answers: 417

Document 1:
Question: What are the most common foods in Estonia?...
Answer: 'black bread, pork, potatoes, and dairy products.'
Has answer: True

Document 2:
Question: What was the medieval flute made from?...
Answer: 'wood'
Has answer: True

Document 3:
Question: What was used to set the year number?...
Answer: (empty)
Has answer: False

Document 4:
Question: Where did the imperial family live?...
Answer: 'Kyoto'
Has answer: True


In [11]:
type(embedding_documents)

list

##### Hybrid Search

In [12]:
import os
import uuid

from qdrant_client import models, QdrantClient
from fastembed import TextEmbedding
from groq import Groq
from dotenv import load_dotenv

/home/jaykold/dev/llm-project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-17 17:49:53.751378082 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card0/device/vendor"


In [13]:
load_dotenv()

ENBEDDING_DIMENSIONALITY=512
GROQ_API_KEY = os.getenv("GROQ_API_KEY1")

In [14]:
for model in TextEmbedding.list_supported_models():
    if model['dim'] >= ENBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-base-en",
  "sources": {
    "hf": "Qdrant/fast-bge-base-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.42,
  "additional_files": [],
  "dim": 768,
  "tasks": {}
}
{
  "model": "BAAI/bge-base-en-v1.5",
  "sources": {
    "hf": "qdrant/bge-base-en-v1.5-onnx-q",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.21,
  "additional_files": [],
  "dim": 768,
  

In [15]:
dense_model="jinaai/jina-embeddings-v2-small-en"
sparse_model="Qdrant/bm25"

In [26]:
client = QdrantClient("http://localhost", port=6333)

In [27]:
collection_name="squad_rag"

# Create a collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config= {
        "jina-small" : models.VectorParams(
            size=ENBEDDING_DIMENSIONALITY,
            distance=models.Distance.COSINE
        )
    },
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF
        )
    }
)

UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `squad_rag` already exists!"},"time":0.001959297}'

In [24]:
os.environ.pop('http_proxy', None)
os.environ.pop('https_proxy', None)
os.environ.pop('HTTP_PROXY', None)
os.environ.pop('HTTPS_PROXY', None)


'http://trendgate.interswitchng.com:8080'

In [28]:
#client.delete_collection(collection_name)
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='squad_rag')])

In [29]:
embedding_documents

[{'id': '56e0f3907aa994140058e80a',
  'text': 'Question: What term characterizes the intersection of the rites with the Roman Catholic Church? Context: The Roman Catholic Church canon law also includes the main five rites (groups) of churches which are in full union with the Roman Catholic Church and the Supreme Pontiff:',
  'metadata': {'title': 'Canon law',
   'context': 'The Roman Catholic Church canon law also includes the main five rites (groups) of churches which are in full union with the Roman Catholic Church and the Supreme Pontiff:',
   'question': 'What term characterizes the intersection of the rites with the Roman Catholic Church?',
   'answer': 'full union',
   'has_answer': True}},
 {'id': '571adcf932177014007e9f56',
  'text': "Question: What was Alexandria known for? Context: Alexandria was the most important trade center in the whole empire during Athanasius's boyhood. Intellectually, morally, and politically—it epitomized the ethnically diverse Graeco-Roman world, eve

In [84]:
points = []

for record in embedding_documents:

        point = models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "bm25": models.Document(
                    text=record["text"],
                    model=sparse_model,
                ),
                "jina-small": models.Document(
                    text=record["text"],
                    model=dense_model
                )
            },
            payload={
                "metadata": record["metadata"]
            }
        )
        points.append(point)

In [51]:
display(points[5:7])

[PointStruct(id='47e43163f10143d98d0a8b02e20dac03', vector={'bm25': Document(text='Question: What was used to set the year number? Context: Prior to 1917, Turkey used the lunar Islamic calendar with the Hegira era for general purposes and the Julian calendar for fiscal purposes. The start of the fiscal year was eventually fixed at 1 March and the year number was roughly equivalent to the Hegira year (see Rumi calendar). As the solar year is longer than the lunar year this originally entailed the use of "escape years" every so often when the number of the fiscal year would jump. From 1 March 1917 the fiscal year became Gregorian, rather than Julian. On 1 January 1926 the use of the Gregorian calendar was extended to include use for general purposes and the number of the year became the same as in other countries.', model='Qdrant/bm25', options=None), 'jina-small': Document(text='Question: What was used to set the year number? Context: Prior to 1917, Turkey used the lunar Islamic calenda

In [52]:
embedding_documents[5]

{'id': '5a36fffb95360f001af1b3c1',
 'text': 'Question: What was used to set the year number? Context: Prior to 1917, Turkey used the lunar Islamic calendar with the Hegira era for general purposes and the Julian calendar for fiscal purposes. The start of the fiscal year was eventually fixed at 1 March and the year number was roughly equivalent to the Hegira year (see Rumi calendar). As the solar year is longer than the lunar year this originally entailed the use of "escape years" every so often when the number of the fiscal year would jump. From 1 March 1917 the fiscal year became Gregorian, rather than Julian. On 1 January 1926 the use of the Gregorian calendar was extended to include use for general purposes and the number of the year became the same as in other countries.',
 'metadata': {'title': 'Gregorian calendar',
  'context': 'Prior to 1917, Turkey used the lunar Islamic calendar with the Hegira era for general purposes and the Julian calendar for fiscal purposes. The start of 

In [85]:
client.upload_points(
    collection_name=collection_name,
    points=points,
)

In [35]:
def rrf_search(query:str, limit:int=2) -> list[models.ScoredPoint]:
    results = client.query_points(
        collection_name=collection_name,
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model=dense_model
                ),
                using="jina-small",
                limit=(3 * limit),
            ),
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model=sparse_model
                ),
                using="bm25",
                limit=(3 * limit)
            )
        ],

        # Fusion using Reciprocal Rank Fusion (RRF)
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        with_payload=True,
        limit=limit,
    )

    return results.points

In [31]:
documents[10]['question']

'What were many pieces of secular art carved from in this period?'

In [32]:
results = rrf_search(documents[40]['question'])

print(json.dumps(embedding_documents[40], indent=2))

print(results[0].payload['metadata'])

Fetching 18 files: 100%|██████████| 18/18 [00:01<00:00,  9.46it/s]


{
  "id": "570aa3634103511400d598ac",
  "text": "Question: When did Houston start using light rail lines? Context: METRO began light rail service on January 1, 2004, with the inaugural track (\"Red Line\") running about 8 miles (13 km) from the University of Houston\u2013Downtown (UHD), which traverses through the Texas Medical Center and terminates at NRG Park. METRO is currently in the design phase of a 10-year expansion plan that will add five more lines. and expand the current Red Line. Amtrak, the national passenger rail system, provides service three times a week to Houston via the Sunset Limited (Los Angeles\u2013New Orleans), which stops at a train station on the north side of the downtown area. The station saw 14,891 boardings and alightings in fiscal year 2008. In 2012, there was a 25 percent increase in ridership to 20,327 passengers embarking from the Houston Amtrak station.",
  "metadata": {
    "title": "Houston",
    "context": "METRO began light rail service on January 

In [ ]:
def multi_stage_search(query: str, limit: int = 2) -> list[models.ScoredPoint]:
    results = client.query_points(
        collection_name=collection_name,
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model=dense_model
                ),
                using="jina-small",
                limit=(2 * limit),
            )
        ],
        query=models.Document(
            text=query,
            model=sparse_model,
        ),
        using="bm25",
        limit=limit,
        with_payload=True,
    )

    return results.points

##### RAG

In [105]:
def build_prompt(query, search_results:models.ScoredPoint) -> str:
   
    context = "\n\n".join(
        [f"{'Context:'.ljust(10)}{res.payload['metadata']['context']}\n"
        f"{'Question:'.ljust(10)}{res.payload['metadata']['question']}\n"
        f"{'Answer:'.ljust(10)}{res.payload['metadata']['answer']}\n"
        f"{'Has_answer:'.ljust(10)}{res.payload['metadata']['has_answer']}"
        for res in search_results]
    )

    prompt = [
        {
            "role": "system",
            "content": """You are a precise and reliable assistant. Your goal is to answer the question using *only* the information provided in the CONTEXT. Do not guess or make up any information.
            Instructions:
            - If the `Has_answer` flag is False, check the context carefully for relevant details. If no answer is clearly supported, reply with: "I don't know. This is beyong my knowledge base."
            - If the answer *is* supported in the context, rephrase and present it clearly and concisely, ensuring it is directly grounded in the information provided.
            - Do not use any external knowledge or assumptions. Stick strictly to the *CONTEXT*.
            - Do not fabricate answers or speculate beyond the given information."""
        },
        {
            "role": "user",
            "content": f"QUESTION:\n {query}\n CONTEXT:\n {context.strip()}"
        }
    ]

    return prompt

In [106]:
query = documents[25]['question']
search_results = rrf_search(query, limit=1)
prompt = build_prompt(query, search_results)

In [146]:
documents[28]

{'id': '56e19227cd28a01900c679d2',
 'title': 'Catalan language',
 'context': 'In Western Catalan, unstressed vowels reduce to five: /e ɛ/ > [e]; /o ɔ/ > [o]; /a u i/ remain distinct. This reduction pattern, inherited from Proto-Romance, is also found in Italian and Portuguese. Some Western dialects present further reduction or vowel harmony in some cases.',
 'question': 'Which letters remain distinct?',
 'answer': '/a u i/'}

In [109]:
print(prompt)

[{'role': 'system', 'content': 'You are a precise and reliable assistant. Your goal is to answer the question using *only* the information provided in the CONTEXT. Do not guess or make up any information.\n            Instructions:\n            - If the `Has_answer` flag is False, check the context carefully for relevant details. If no answer is clearly supported, reply with: "I don\'t know. This is beyong my knowledge base."\n            - If the answer *is* supported in the context, rephrase and present it clearly and concisely, ensuring it is directly grounded in the information provided.\n            - Do not use any external knowledge or assumptions. Stick strictly to the *CONTEXT*.\n            - Do not fabricate answers or speculate beyond the given information.'}, {'role': 'user', 'content': 'QUESTION:\n What provides critical support for drug discovery and the availability of economic resources?\n CONTEXT:\n Context:  Biodiversity provides critical support for drug discovery a

In [ ]:
def llm_response(prompt, model: str = "openai/gpt-oss-20b") -> str:
    client = Groq(api_key=GROQ_API_KEY)
    response = client.chat.completions.create(messages=prompt, model=model)

    return response.choices[0].message.content

In [143]:
print(llm_response(prompt))

I don't know. This is beyond my knowledge base.
